In [1]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import normalize
from sklearn.metrics import cohen_kappa_score
from formfyxer import lit_explorer
import pingouin as pg
import requests
import shutil

/home/brycew/Developer/LITLab/litlab_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [26]:
all_expert = pd.read_excel("RateMyPDF Individual Form Expert Benchmarks(1-176).xlsx")
name_column = "Full name"
good_column="good_column"
rating_column = 'ratings'
all_expert = all_expert.rename(columns={
  "What form are you scoring? Please include the full URL, like: https://courtformsonline.org/forms/6e420f1b3575cfd8ef94b71977da9e38252e3395a78439709c760de4.pdf\n": "form_url",
  "From 1-5, with 1 being the easiest and 5 being the hardest, how complex do you think this form is?\n ": rating_column,
  "From 1-5 stars, with 5 being the best, how good a form do you think this is? Use any criteria that make sense to you.\n": good_column,
})
form_column = 'form_name'
all_expert[form_column] = all_expert['form_url'].apply(lambda y: y.split("/")[-1].strip())

Index(['ID', 'Full name', 'form_url',
       'Using your own judgment, what do you think that the minimum reading grade level for this document is?\n',
       'What is your best guess as to how long it would take a typical litigant to complete this form accurately?\n',
       'The fields are organized so that similar topics are adjacent to each other',
       'The number of fields on each page is about right',
       'Most litigants can read and understand the prompts and instructions on this form',
       'Most litigants can accurately fill in this form without help from an attorney',
       'Most litigants can provide a complete answer to the questions on this form',
       'Most litigants can provide an answer to the questions on this form without having to ask a third party',
       'Is there a page that is radically different from the other pages on the form such that your answers above do not apply to each page? Which page and why?',
       'Does this form require the litigant to

In [36]:
df = all_expert[[form_column, rating_column, name_column, good_column]]
def normalize_df(df, group_by_column, normalize_column, output_column):
  mean_and_stddev = (
      df.groupby(group_by_column)[normalize_column]
      .agg(["mean", "std"])
      .rename(columns={"mean": f"{normalize_column}_mean", "std": f"{normalize_column}_stddev"})
      .reset_index()
  )
  df = pd.merge(df, mean_and_stddev, on=group_by_column)
  df[output_column] = (df[normalize_column] - df[f"{normalize_column}_mean"]) / df[f"{normalize_column}_stddev"]
  
mean_and_stddev = (
    df.groupby(name_column)[rating_column]
    .agg(["mean", "std"])
    .rename(columns={"mean": "reviewer_mean", "std": "reviewer_stddev"})
    .reset_index()
)
df = pd.merge(df, mean_and_stddev, on=name_column)
df["z_score"] = (df[rating_column] - df["reviewer_mean"]) / df["reviewer_stddev"]

mean_and_stddev_good = (
    df.groupby(name_column)[good_column]
    .agg(["mean", "std"])
    .rename(columns={"mean": "reviewer_good_mean", "std": "reviewer_good_stddev"})
    .reset_index()
)
df = pd.merge(df, mean_and_stddev_good, on=name_column)
df["z_good_score"] = (df[good_column] - df["reviewer_good_mean"]) / df["reviewer_good_stddev"]
sorted_experts = (
    df.groupby("form_name")
    .mean()
    .sort_values(by="z_score", ascending=True)
    .reset_index()
)
print(df.keys())
# print(pd.pivot_table(df[[name_column, 'form_name', rating_column]], index=name_column, columns='form_name').T)
# print(df.groupby('form_name').)

Index(['form_url', 'ratings', 'Full name', 'good_column', 'form_name',
       'reviewer_mean', 'reviewer_stddev', 'z_score', 'reviewer_good_mean',
       'reviewer_good_stddev', 'z_good_score'],
      dtype='object')


/tmp/ipykernel_3691754/2652558115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["form_name"] = df[form_column].apply(lambda y: y.split("/")[-1].strip())
/tmp/ipykernel_3691754/2652558115.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("form_name")


In [39]:
def calc_score(fname):
    # full_url = "https://courtformsonline.org/forms/" + fname
    # local_filename = "/tmp/" + fname
    # with requests.get(full_url, stream=True) as r:
    #  with open(local_filename, 'wb') as f:
    #    shutil.copyfileobj(r.raw, f)
    # The already downloaded and re-formated field PDFs, the ones that the stats are from in the paper.
    local_filename = "../../../tmp3/" + fname
    stats = lit_explorer.parse_form(local_filename)
    return lit_explorer.form_complexity(stats)


sorted_experts["our_complexity_score"] = sorted_experts["form_name"].apply(calc_score)

Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Sentences...
Starting to find passives...
Detecting Se

In [40]:
sorted_experts["z_complexity_score"] = (
    sorted_experts["our_complexity_score"]
    - np.mean(sorted_experts["our_complexity_score"])
) / np.std(sorted_experts["our_complexity_score"])

In [63]:
sorted_experts_copy = sorted_experts.copy()
sorted_experts_copy[name_column] = "Algo"
sorted_experts_copy["z_score"] = sorted_experts_copy["z_complexity_score"]
df_algo = df.copy()
random_df = pd.DataFrame()
import random
random_df['z_score'] = (np.array(random.sample(range(0, 1000), len(sorted_experts_copy))) - 500) / 250
random_df[name_column] = 'random'
random_df['form_name'] = sorted_experts['form_name']
df_algo_grader = pd.concat([df_algo, sorted_experts_copy])
df_random_grader = pd.concat([df_algo, random_df])

bins = [-2, -1, 0, 1, 2, 3]
df["z_binned"] = np.digitize(df["z_score"], bins=bins)
df_algo_grader["z_binned"] = np.digitize(df_algo_grader["z_score"], bins=bins)
df_random_grader["z_binned"] = np.digitize(df_random_grader["z_score"], bins=bins)
results_complex = pg.intraclass_corr(data=df, targets='form_name', raters=name_column, ratings="z_binned", nan_policy='omit')
results_z_complex_with_us = pg.intraclass_corr(data=df_algo_grader, targets='form_name', raters=name_column, ratings="z_binned", nan_policy='omit')
results_z_complex_with_random = pg.intraclass_corr(data=df_random_grader, targets='form_name', raters=name_column, ratings="z_binned", nan_policy='omit')

#print(df_algo_grader.groupby(name_column).mean())
print(results_complex)
print(results_z_complex_with_us)
print(results_z_complex_with_random)

    Type              Description       ICC         F  df1  df2      pval  \
0   ICC1   Single raters absolute  0.318245  3.800813    4   25  0.015098   
1   ICC2     Single random raters  0.320442  3.912134    4   20  0.016651   
2   ICC3      Single fixed raters  0.326761  3.912134    4   20  0.016651   
3  ICC1k  Average raters absolute  0.736898  3.800813    4   25  0.015098   
4  ICC2k    Average random raters  0.738854  3.912134    4   20  0.016651   
5  ICC3k     Average fixed raters  0.744385  3.912134    4   20  0.016651   

          CI95%  
0  [0.02, 0.84]  
1  [0.03, 0.84]  
2  [0.02, 0.84]  
3  [0.12, 0.97]  
4  [0.14, 0.97]  
5   [0.1, 0.97]  
    Type              Description       ICC         F  df1  df2      pval  \
0   ICC1   Single raters absolute  0.402108  5.707792    4   30  0.001540   
1   ICC2     Single random raters  0.404255  5.959322    4   24  0.001775   
2   ICC3      Single fixed raters  0.414683  5.959322    4   24  0.001775   
3  ICC1k  Average raters a

In [15]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required
# to sort an array when only swapping of adjacent
# elements is allowed.
# include <bits/stdc++.h>


# This function merges two sorted arrays and returns inversion count in the arrays.*/
def merge(arr, temp, left, mid, right):
    inv_count = 0

    i = left  # i is index for left subarray*/
    j = mid  # i is index for right subarray*/
    k = left  # i is index for resultant merged subarray*/
    while (i <= mid - 1) and (j <= right):
        if arr[i] <= arr[j]:
            temp[k] = arr[i]
            k += 1
            i += 1
        else:
            temp[k] = arr[j]
            k += 1
            j += 1

            # this is tricky -- see above explanation/
            # diagram for merge()*/
            inv_count = inv_count + (mid - i)

    # Copy the remaining elements of left subarray
    # (if there are any) to temp*/
    while i <= mid - 1:
        temp[k] = arr[i]
        k += 1
        i += 1

    # Copy the remaining elements of right subarray
    # (if there are any) to temp*/
    while j <= right:
        temp[k] = arr[j]
        k += 1
        j += 1

    # Copy back the merged elements to original array*/
    for i in range(left, right + 1, 1):
        arr[i] = temp[i]

    return inv_count


# An auxiliary recursive function that sorts the input
# array and returns the number of inversions in the
# array. */
def _mergeSort(arr, temp, left, right):
    inv_count = 0
    if right > left:
        # Divide the array into two parts and call
        # _mergeSortAndCountInv()
        # for each of the parts */
        mid = int((right + left) / 2)

        # Inversion count will be sum of inversions in
        # left-part, right-part and number of inversions
        # in merging */
        inv_count = _mergeSort(arr, temp, left, mid)
        inv_count += _mergeSort(arr, temp, mid + 1, right)

        # Merge the two parts*/
        inv_count += merge(arr, temp, left, mid + 1, right)

    return inv_count


# This function sorts the input array and returns the
# number of inversions in the array */
def countSwaps(arr):
    n = len(arr)
    temp = [0 for i in range(n)]
    return _mergeSort(arr, temp, 0, n - 1)  # Time to get random sorts

In [16]:
# Time to get sorts
def get_avg_sort_count(df, final_order_col, sort_by, runs=1):
    swap_counts = []
    for i in range(runs):
        if sort_by == "random":
            array = df[final_order_col].sample(frac=1).values.copy()
        else:
            df = df.sample(frac=1).sort_values(by=sort_by, ascending=True)
            array = df[final_order_col].values.copy()
        swap_counts.append(countSwaps(array))
    return np.array(swap_counts).mean()


print(
    f"Random sort count: {get_avg_sort_count(sorted_experts, 'z_score', 'random', runs=5000)}"
)
print(
    f"Machine sort count: {get_avg_sort_count(sorted_experts, 'z_score', 'z_complexity_score', runs=5000)}"
)

Random sort count: 388.256
Machine sort count: 306.0


In [17]:
# Time to get cohen's kappa
# put both ours and theirs back into bins.
bins = [-3, -2, -1, 0, 1, 2, 3]
expert_bins = np.digitize(sorted_experts["z_score"], bins=bins)
algo_bins = np.digitize(sorted_experts["z_complexity_score"], bins=bins)
print(
    f'scipy_cohen_kappa: {cohen_kappa_score(expert_bins, algo_bins, weights="quadratic")}'
)
# print(stats.ttest_rel())

scipy_cohen_kappa: 0.6160267111853088


In [57]:
bins = [-2, -1, 0, 1, 2, 3]
expert_bins = np.digitize(sorted_experts["z_score"], bins=bins)
algo_bins = np.digitize(sorted_experts["z_complexity_score"], bins=bins)

new_df = pd.DataFrame([], columns=["reviewer", "score"])
new_df_algo = new_df.copy()
new_df_algo["score"] = algo_bins
new_df_algo["reviewer"] = "algo"
new_df_algo["idx"] = new_df_algo.index
new_df_expert = new_df.copy()
new_df_expert["score"] = expert_bins
new_df_expert["reviewer"] = "expert"
new_df_expert["idx"] = new_df_expert.index
new_df = pd.concat([new_df_algo, new_df_expert])

df["z_binned"] = np.digitize(df["z_score"], bins=bins)
results_complex = pg.intraclass_corr(data=df, targets='form_name', raters=name_column, ratings=rating_column, nan_policy='omit')
results_good = pg.intraclass_corr(data=df, targets='form_name', raters=name_column, ratings=good_column, nan_policy='omit')
results_z_good = pg.intraclass_corr(data=df, targets='form_name', raters=name_column, ratings='z_good_score', nan_policy='omit')
results_z_complex = pg.intraclass_corr(data=df, targets='form_name', raters=name_column, ratings="z_binned", nan_policy='omit')

results_complex = pg.intraclass_corr(data=new_df, targets="idx", raters="reviewer", ratings="score")

print(results_z_complex)
print(results_z_complex_with_us)
print(results_complex)

    Type              Description       ICC         F  df1  df2      pval  \
0   ICC1   Single raters absolute  0.318245  3.800813    4   25  0.015098   
1   ICC2     Single random raters  0.320442  3.912134    4   20  0.016651   
2   ICC3      Single fixed raters  0.326761  3.912134    4   20  0.016651   
3  ICC1k  Average raters absolute  0.736898  3.800813    4   25  0.015098   
4  ICC2k    Average random raters  0.738854  3.912134    4   20  0.016651   
5  ICC3k     Average fixed raters  0.744385  3.912134    4   20  0.016651   

          CI95%  
0  [0.02, 0.84]  
1  [0.03, 0.84]  
2  [0.02, 0.84]  
3  [0.12, 0.97]  
4  [0.14, 0.97]  
5   [0.1, 0.97]  
    Type              Description       ICC         F  df1  df2      pval  \
0   ICC1   Single raters absolute  0.402108  5.707792    4   30  0.001540   
1   ICC2     Single random raters  0.404255  5.959322    4   24  0.001775   
2   ICC3      Single fixed raters  0.414683  5.959322    4   24  0.001775   
3  ICC1k  Average raters a

In [34]:
df.groupby(name_column).mean()

/tmp/ipykernel_3691754/3204406776.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(name_column).mean()


,ratings,good_column,reviewer_mean,reviewer_stddev,z_score,z_binned
Full name,,,,,,
Reviewer 1,3.200000,2.066667,3.200000,0.484234,-3.441691e-16,2.400000
Reviewer 2,2.333333,3.517241,2.333333,1.061337,-1.480297e-16,2.333333
Reviewer 3,1.742857,4.257143,1.742857,1.010034,-2.537653e-17,2.742857
Reviewer 4,2.680000,3.200000,2.680000,1.107550,-1.643130e-16,2.680000
Reviewer 5,1.914286,3.342857,1.914286,0.981338,-7.612958e-17,2.914286
Reviewer 6,1.850000,3.200000,1.850000,1.136708,-5.551115e-17,2.750000
